# Reversing Game of Life 

The rules for the Game of Life are: 
- Any live cell with fewer than two live neighbors dies, as if by underpopulation.
- Any live cell with two or three live neighbors lives on to the next generation.
- Any live cell with more than three live neighbors dies, as if by overpopulation.
- Any dead cell with exactly three live neighbors becomes a live cell, as if by reproduction.

In [46]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from scipy.signal import convolve2d
import numpy as np
from scipy.ndimage import convolve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error

# Implementation of the game of life

In [ ]:

# Initialize the grid with random alive (1) and dead (0) cells
def initialize_grid(size):
    return np.random.choice([0, 1], size*size, p=[0.8, 0.2]).reshape(size, size)

# Define the function to update the grid based on the Game of Life rules
def update_grid(grid):
    new_grid = grid.copy()
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            # Calculate the number of live neighbors
            live_neighbors = np.sum(grid[max(0, i-1):min(i+2, grid.shape[0]), max(0, j-1):min(j+2, grid.shape[1])]) - grid[i, j]
            
            # Apply the Game of Life rules
            if grid[i, j] == 1:
                if live_neighbors < 2 or live_neighbors > 3:
                    new_grid[i, j] = 0
            else:
                if live_neighbors == 3:
                    new_grid[i, j] = 1
    return new_grid

# Initialize the grid
size = 20  # Grid size
grid = initialize_grid(size)

# Create output directory for frames
output_dir = 'game_of_life_frames/10steps'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Plot and save each state
def save_grid(grid, step):
    plt.imshow(grid, interpolation='nearest', cmap='binary')
    plt.title(f"Step {step}")
    plt.savefig(f"{output_dir}/frame_{step:03d}.png")
    plt.close()

# Number of steps to simulate
steps = 10

# Save the initial state
save_grid(grid, 0)

# Simulate and save each step
for step in range(1, steps + 1):
    grid = update_grid(grid)
    save_grid(grid, step)

print(f"Saved {steps + 1} frames in {output_dir}")

In [3]:
def create_video_from_frames(output_dir, video_name, frame_rate):
    # Get list of frame files sorted by name
    frame_files = sorted([f for f in os.listdir(output_dir) if f.endswith('.png')])
    
    # Read the first frame to get dimensions
    first_frame = cv2.imread(os.path.join(output_dir, frame_files[0]))
    height, width, layers = first_frame.shape

    # Define the codec and create VideoWriter object
    video_path = os.path.join(output_dir, video_name)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can use other codecs like 'XVID'
    video = cv2.VideoWriter(video_path, fourcc, frame_rate, (width, height))

    for frame_file in frame_files:
        frame = cv2.imread(os.path.join(output_dir, frame_file))
        video.write(frame)

    video.release()
    print(f"Video saved at {video_path}")

output_dir = 'game_of_life_frames/10steps'
video_name = 'game_of_life.mp4'
frame_rate = 10  # frames per second

create_video_from_frames(output_dir, video_name, frame_rate)

Video saved at game_of_life_frames/10steps/game_of_life.mp4


# Preprocessing step

In [4]:
train_csv = pd.read_csv('/Users/victorgutierrezgarcia/Desktop/MUTSC/2º Cuatri /BILN/biln_project/bioinspired_learning/data/train.csv')
train_csv.head()

,id,delta,start.1,start.2,start.3,start.4,start.5,start.6,start.7,start.8,...,stop.391,stop.392,stop.393,stop.394,stop.395,stop.396,stop.397,stop.398,stop.399,stop.400
0,1,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,5,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,3,2,0,0,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,4,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,4,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [11]:
def preprocessing_lattice(dataframe: pd.DataFrame, min_delta: int) -> pd.DataFrame:
    preprocessed_df = []
    for index, row in dataframe.iterrows(): 
        delta = row['delta']
        if delta < min_delta: 
            continue
        preprocessed_df.append(row)
    
    return pd.DataFrame(preprocessed_df)

In [15]:
high_life_time = preprocessing_lattice(train_csv, 4) 

In [69]:
M = ((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1))

def calc_neighs(field, i, j):
    """ Calculate number of neighbors alive (assuming square field) """
    neighs = 0
    n = len(field)
    for m in M:
        row_idx = m[0] + i
        col_idx = m[1] + j
        if 0 <= row_idx < n and 0 <= col_idx < n:
            if field[row_idx][col_idx]:
                neighs += 1
    return neighs

def make_move(field, moves=1, bool_isolated_cells = False):
    """ Make a move forward according to Game of Life rules """
    n = len(field)
    cur_field = field
    isolated_cells = 0 # This variable will take into account the cells that will die in the next iteration 
    for _ in range(moves):
        new_field = np.zeros((n, n), dtype='uint8')
        for i in range(n):
            for j in range(n):
                neighs = calc_neighs(cur_field, i, j)
                if neighs < 2 or neighs > 3: 
                    isolated_cells += 1
                if cur_field[i][j] and neighs == 2:
                    new_field[i][j] = 1
                if neighs == 3:
                    new_field[i][j] = 1
        cur_field = new_field
    if bool_isolated_cells:
        return cur_field, isolated_cells
    else: 
        return cur_field

def generate_population(size, random_state=-1):
    """
    Generating initial population of individual solutions
    :return: initial population as a list of 20x20 arrays
    """
    if random_state != -1:
        np.random.seed(random_state)
    initial_states = np.split(np.random.binomial(1, 0.5, (20 * size, 20)).astype('uint8'), size)
    return [make_move(state, 5) for state in initial_states]

def fitness(start_field, end_field, delta):
    """
    Calculate fitness for particular candidate (start configuration of the field)
    :param start_field: candidate (start configuration)
    :param end_field: target (stop configuration)
    :param delta: number of steps to proceed before comparing to stop configuration
    :return: value in range [0, 1] that indicates fractions of cells that match their state
    """
    candidate, isolated_cells = make_move(start_field, moves=delta, bool_isolated_cells=True)
    return (candidate == end_field).sum() / 400
      
def count_underpopulated_cells(board: np.arrays) -> int:
    """
    Count cells that will die due to underpopulation (having 1 or 2 neighbors).
    :param board: Current board configuration
    :return: Number of underpopulated cells
    """
    # Define the kernel to count neighbors
    kernel = np.array([[1, 1, 1],
                       [1, 0, 1],
                       [1, 1, 1]])
    
    neighbor_count = convolve2d(board, kernel, mode='same', boundary='wrap')
    # Underpopulated cells: live cells with 1 or 2 neighbors
    underpopulated = (board == 1) & ((neighbor_count == 1) | (neighbor_count == 2))
    
    return np.sum(underpopulated)

def fitness_with_penalization(start_field: np.array, end_field: np.array, delta: int, penalty_factor = 0.1) -> int:
    """
    Calculate fitness for a candidate with penalization for underpopulated cells.
    :param start_field: Candidate start configuration
    :param end_field: Target end configuration
    :param delta: Number of steps to evolve before comparing to end configuration
    :param penalty_factor: Hyperparameter to adjust the penalization effect
    :return: Fitness value adjusted for penalization
    """
    # Simulate the next state (after delta steps)
    candidate, isolated_cells = make_move(start_field, moves=delta, bool_isolated_cells=True)
    
    # Calculate the initial fitness based on the target configuration
    match_fraction = (candidate == end_field).sum() / (start_field.shape[0] * start_field.shape[1])
    
    # Count the underpopulated cells in the initial configuration
    underpopulated_cells = count_underpopulated_cells(start_field)
    
    # Apply penalization
    penalized_fitness = match_fraction - (penalty_factor * underpopulated_cells / (start_field.shape[0] * start_field.shape[1]))
    
    # Ensure the fitness value remains within [0, 1]
    return max(0, min(penalized_fitness, 1))

def score_population(population, target, delta):
    """
    Apply fitness function for each gene in a population
    :param population: list of candidate solutions
    :param target: 20x20 array that represents field in stopping condition
    :param delta: number of steps to revert
    :return: list of scores for each solution
    """
    # return [fitness(gene, target, delta) for gene in population]
    return [fitness_with_penalization(gene, target, delta) for gene in population]

def selection(population, scores, retain_frac=0.8, retain_random=0.05):
    """
    Apply selection operator to the population
    :param population: list of candidate solutions
    :param scores: list of score associated with each individual
    :param retain_frac: percent of top individuals to retain
    :param retain_random: chance of retaining sub-optimal individuals in the population
    """
    retain_len = int(len(scores) * retain_frac)
    sorted_indices = np.argsort(scores)[::-1]
    population = [population[idx] for idx in sorted_indices]
    selected = population[:retain_len]
    leftovers = population[retain_len:]

    for gene in leftovers:
        if np.random.rand() < retain_random:
            selected.append(gene)
    return selected


def mutate(field, switch_frac=0.1):
    """ Inplace mutation of the provided field """
    a = np.random.binomial(1, switch_frac, size=(20, 20)).astype('bool')
    field[a] += 1
    field[a] %= 2
    return field


def crossover(mom, dad):
    """ Take two parents, return two children, interchanging half of the allels of each parent randomly """
    select_mask = np.random.binomial(1, 0.5, size=(20, 20)).astype('bool')
    child1, child2 = np.copy(mom), np.copy(dad)
    child1[select_mask] = dad[select_mask]
    child2[select_mask] = mom[select_mask]
    return child1, child2


def evolve(population, target, delta, retain_frac=0.8, retain_random=0.05, mutate_chance=0.05):
    """
    Evolution step
    :param population: list or candidate solutions
    :param target: 20x20 array that represents field in stopping condition
    :param delta: number of steps to revert
    :param retain_frac: percent of top individuals to proceed into the next genetation
    :param retain_random: chance of retaining sub-optimal individual
    :param mutate_chance: chance of mutating the particular individual
    :return: new generation of the same size
    """
    scores = score_population(population, target, delta)
    next_population = selection(population, scores, retain_frac=retain_frac, retain_random=retain_random)
    
    # mutate everyone expecting for the best candidate
    for gene in next_population[1:]:
        if np.random.rand() < mutate_chance:
            mutate(gene)

    places_left = len(population) - len(next_population)
    children = []
    parent_max_idx = len(next_population) - 1
    while len(children) < places_left:
        mom_idx, dad_idx = np.random.randint(0, parent_max_idx, 2)
        if mom_idx != dad_idx:
            child1, child2 = crossover(next_population[mom_idx], next_population[dad_idx])
            children.append(child1)
            if len(children) < places_left:
                children.append(child2)
    next_population.extend(children)
    return next_population


def solve(target, delta, population_size=200, n_generations=300):
    """
    :param target: 20x20 array that represents field in stopping condition
    :param delta: number of steps to revert
    :param n_generations: number of evolution generations. Overrides initialization value if specified
    :return: 20x20 array that represents the best start field found and associated fitness value
    """
    population = generate_population(population_size, random_state=42)
    for generation in range(n_generations):
        population = evolve(population, target, delta)
        if generation == 0:
            print("Generation #: best score")
        elif generation % 50 == 0:
            print("Generation ", generation, ": ", fitness_with_penalization(population[0], target, delta))
    return population[0]

In [62]:
def save_grid(grid: np.array, step: int, title: str, type: int) -> None:
    plt.imshow(grid, interpolation='nearest', cmap='binary')
    plt.title(f"{title}")
    plt.savefig(f"{output_dir}/frame_{type:03d}_{step}.png")
    plt.close()

def save_plot(grid: np.array, index: int, code: int, title: str) -> None: 
    plt.imshow(grid, interpolation='nearest', cmap='binary')
    plt.title(title)
    save_grid(grid, index, title, code)

# Metrics

In [63]:
def compute_error(start_individual, best_guess) -> int: 
    return (start_individual == best_guess).sum() / 400

def compute_extended_hamming(real_initial: np.array, predicted_initial: np.array) -> int:
    """
    Compute the extended Hamming distance considering local neighborhoods.
    :param real_initial: Real initial configuration (20x20 numpy array)
    :param predicted_initial: Predicted initial configuration (20x20 numpy array)
    :return: Normalized extended Hamming distance
    """
    kernel = np.ones((3, 3), dtype=int)
    
    # Count the number of live cells in the neighborhood (including the cell itself)
    real_neighbors_count = convolve(real_initial, kernel, mode='constant', cval=0)
    predicted_neighbors_count = convolve(predicted_initial, kernel, mode='constant', cval=0)
    
    # Calculate the number of matching cells in the neighborhood
    matching_neighbors = (real_neighbors_count == predicted_neighbors_count).sum()
    
    # Normalize by the total number of cells to get the extended Hamming distance
    extended_hamming_distance = matching_neighbors / real_initial.size
    
    return extended_hamming_distance
    
def evaluate_predictions(real_initial, predicted_initial, end_individual, fwd_prediction, delta):
    """
    Evaluate the predicted initial board against the real initial board.
    
    :param real_initial: Real initial configuration
    :param predicted_initial: Predicted initial configuration
    :param target_configuration: Target configuration after delta steps
    :param delta: Number of steps to evolve to reach target configuration
    :return: Dictionary of evaluation metrics
    """
    # Accuracy
    accuracy = accuracy_score(real_initial.flatten(), predicted_initial.flatten())
    
    # Precision, Recall, F1 Score
    precision = precision_score(real_initial.flatten(), predicted_initial.flatten())
    recall = recall_score(real_initial.flatten(), predicted_initial.flatten())
    f1 = f1_score(real_initial.flatten(), predicted_initial.flatten())
    
    # Mean Squared Error
    mse = mean_squared_error(real_initial.flatten(), predicted_initial.flatten())
    
    # Evolution Consistency
    evolution_consistency = accuracy_score(end_individual.flatten(), fwd_prediction.flatten())
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'mse': mse,
        'evolution_consistency': evolution_consistency
    }

def classification_report_to_dict(report):
    """
    Convert classification report from string to dictionary format.
    :param report: classification report string
    :return: dictionary
    """
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split()
        row['class'] = row_data[0]
        row['precision'] = float(row_data[1])
        row['recall'] = float(row_data[2])
        row['f1_score'] = float(row_data[3])
        row['support'] = int(row_data[4])
        report_data.append(row)
    return pd.DataFrame.from_dict(report_data)


# Main Loop 

In [71]:
target_solutions = []

for index, row in high_life_time.iterrows(): 
    output_dir = 'game_of_life_frames/row_alterntive_fitness_v3_' + str(index)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    id = row['id']
    delta = row['delta']

    start_values = train_csv.iloc[index, 2:402].values
    start_individual = start_values.reshape(20,20)
    save_plot(start_individual, index, 1, title="Start Individual")

    stop_values = train_csv.iloc[index, 402:].values
    end_individual = stop_values.reshape(20,20)
    save_plot(end_individual, index, 2, title="End Individual")

    best_population = solve(end_individual, delta, n_generations=200)
    save_plot(best_population, index, 3, title="Best guess Individual")

    print(f'The error between the initial grid and the guess is{compute_error(start_individual, best_population)}') #El número de celulas que difieren. 

    for delta in range(1, steps + 1):
        guess_fwd = update_grid(best_population)
        
    save_grid(guess_fwd, index, "Forward initial guess grid", 4)
    error_value = compute_error(end_individual, guess_fwd)
    hamming_distance = compute_extended_hamming(start_individual, best_population)
    print(f'The error between the forward guess initial board and the final board from dataset is: {error_value}') #compara el error entre la prediccion propagada y la final dada en el dataset. 
    print(f'The Hamming distance between the two boards is: {hamming_distance}')
    
    classification_report = evaluate_predictions(start_individual, best_population, end_individual, guess_fwd, delta)

    print(f'An extended classification report: {evaluate_predictions(start_individual, best_population, end_individual, guess_fwd, delta)}')
        
    with pd.ExcelWriter(output_dir + '/evaluation_metrics.xlsx') as writer:
            # Save basic metrics
        data = {
                'Metric': ['Error between forward guess and final board', 'Hamming distance between two boards',
                        'Accuracy', 'Precision', 'Recall', 'F1 Score', 'Mean Squared Error', 'Evolution Consistency'],
                'Value': [error_value, hamming_distance,
                        classification_report['accuracy'], classification_report['precision'], classification_report['recall'], classification_report['f1_score'],
                        classification_report['mse'], classification_report['evolution_consistency']]
            }
        df = pd.DataFrame(data)
        df.to_excel(writer, sheet_name='Metrics', index=False)
        

Generation #: best score
Generation  50 :  0.9785
Generation  100 :  0.98925
Generation  150 :  0.9965
The error between the initial grid and the guess is0.9575
The error between the forward guess initial board and the final board from dataset is: 0.9975
The Hamming distance between the two boards is: 0.7375
An extended classification report: {'accuracy': 0.9575, 'precision': 0.21052631578947367, 'recall': 0.6666666666666666, 'f1_score': 0.32, 'mse': 0.0425, 'evolution_consistency': 0.9975}
Generation #: best score
Generation  50 :  0.9480000000000001
Generation  100 :  0.9650000000000001
Generation  150 :  0.97475
The error between the initial grid and the guess is0.87
The error between the forward guess initial board and the final board from dataset is: 0.955
The Hamming distance between the two boards is: 0.3975
An extended classification report: {'accuracy': 0.87, 'precision': 0.11428571428571428, 'recall': 0.16, 'f1_score': 0.13333333333333333, 'mse': 0.13, 'evolution_consistency'

Traceback (most recent call last):
  File "/Users/victorgutierrezgarcia/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/v9/jdh_sf0s6059j5bw53rnglhr0000gn/T/ipykernel_43364/340310337.py", line 19, in <module>
    best_population = solve(end_individual, delta, n_generations=200)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v9/jdh_sf0s6059j5bw53rnglhr0000gn/T/ipykernel_43364/1760325490.py", line 191, in solve
    population = evolve(population, target, delta)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v9/jdh_sf0s6059j5bw53rnglhr0000gn/T/ipykernel_43364/1760325490.py", line 160, in evolve
    scores = score_population(population, target, delta)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v9/jdh_sf0s6059j5bw53rnglhr0000gn/T/ipykernel_43364/1760325490.py", 